lowest 999 doc for each topic

In [9]:
import pandas as pd

# Define the path to your QREL file
data_path = 'D:\VSCODE PROJECT\IR\dataset\input.plt8ah2'

# Define column names for the DataFrame
column_names = ['topicId', 'identifier', 'docId', 'ranking', 'similarityScore', 'systemName']

# Read the QREL file into a DataFrame
df = pd.read_csv(data_path, sep='\t', header=None, names=column_names)

# Display the first few rows of the DataFrame to verify
print(df)


       topicId identifier          docId  ranking  similarityScore systemName
0          401         Q0     FBIS4-9582        0        13.814581    plt8ah2
1          401         Q0    FT932-10861        1        13.342205    plt8ah2
2          401         Q0    FBIS4-18182        2        12.389992    plt8ah2
3          401         Q0     FBIS3-9104        3        12.071641    plt8ah2
4          401         Q0    FT941-15167        4        11.571873    plt8ah2
...        ...        ...            ...      ...              ...        ...
49993      450         Q0    FBIS4-13170      995        10.328714    plt8ah2
49994      450         Q0  LA082190-0080      996        10.327399    plt8ah2
49995      450         Q0  LA123189-0117      997        10.324968    plt8ah2
49996      450         Q0  LA102490-0024      998        10.324179    plt8ah2
49997      450         Q0    FBIS4-11642      999        10.323837    plt8ah2

[49998 rows x 6 columns]


In [10]:
# Count the number of rows per topicid
topicId_counts = df['topicId'].value_counts().sort_index()
print(topicId_counts)

topicId
401    1000
402    1000
403    1000
404    1000
405    1000
406    1000
407    1000
408    1000
409    1000
410    1000
411    1000
412    1000
413    1000
414    1000
415    1000
416     999
417    1000
418    1000
419    1000
420    1000
421    1000
422    1000
423    1000
424    1000
425    1000
426    1000
427    1000
428    1000
429    1000
430    1000
431    1000
432    1000
433    1000
434     999
435    1000
436    1000
437    1000
438    1000
439    1000
440    1000
441    1000
442    1000
443    1000
444    1000
445    1000
446    1000
447    1000
448    1000
449    1000
450    1000
Name: count, dtype: int64


In [11]:
# Check data types of all columns
print("Data type of each column: ")
print(df.dtypes)

# Check null value
print( "Count of null values in each column: ")
print(df.isnull().sum())

Data type of each column: 
topicId              int64
identifier          object
docId               object
ranking              int64
similarityScore    float64
systemName          object
dtype: object
Count of null values in each column: 
topicId            0
identifier         0
docId              0
ranking            0
similarityScore    0
systemName         0
dtype: int64


In [12]:
# Check for duplicates in 'Ranking' within each 'topicId' group
has_duplicates = df.groupby('topicId')['ranking'].apply(lambda x: x.duplicated()).any()

# Check if 'Ranking' is in ascending order within each 'topicId' group
is_ascending_ranking = df.groupby('topicId')['ranking'].apply(lambda x: x.is_monotonic_increasing).all()

# Check if 'SimilarityScore' is in descending order within each 'topicId' group
is_descending_similarity = df.groupby('topicId')['similarityScore'].apply(lambda x: x.is_monotonic_decreasing).all()

# Print the results
print("Ranking has duplicates within each topicId:", has_duplicates)
print("Ranking is in ascending order within each topicId:", is_ascending_ranking)
print("SimilarityScore is in descending order within each topicId:", is_descending_similarity)

Ranking has duplicates within each topicId: False
Ranking is in ascending order within each topicId: False
SimilarityScore is in descending order within each topicId: True


In [13]:
# Function to identify non-increasing ranks within each group
def find_non_increasing_ranks(group):
    # Calculate the difference between consecutive ranks
    rank_diff = group['ranking'].diff()
    # Identify rows where the difference is not positive
    non_increasing = rank_diff[rank_diff <= 0].index
    return non_increasing

# Apply the function to each group and concatenate the results
non_increasing_rows = df.groupby('topicId').apply(find_non_increasing_ranks).reset_index(level=0, drop=True)
non_increasing_rows = non_increasing_rows.explode().dropna().astype(int)

# Display the rows that do not follow an increasing order in ranking
print("Rows with non-increasing ranks:")
print(df.loc[non_increasing_rows])

Rows with non-increasing ranks:
       topicId identifier        docId  ranking  similarityScore systemName
37         401         Q0   FBIS4-9438       36        10.018311    plt8ah2
234        401         Q0  FBIS3-30284      233         8.221656    plt8ah2
527        401         Q0  FBIS3-43174      526         7.473248    plt8ah2
561        401         Q0   FT944-2530      560         7.416789    plt8ah2
660        401         Q0  FBIS3-58045      659         7.255794    plt8ah2
...        ...        ...          ...      ...              ...        ...
49151      450         Q0  FBIS3-52659      152        16.012465    plt8ah2
49362      450         Q0  FBIS3-52615      363        13.305415    plt8ah2
49517      450         Q0  FBIS3-52617      518        12.107282    plt8ah2
49708      450         Q0  FBIS3-12306      709        11.199286    plt8ah2
49805      450         Q0  FBIS3-52611      806        10.906279    plt8ah2

[476 rows x 6 columns]


In [14]:
# Sort the DataFrame by topicid and similarityscore in descending order
df = df.sort_values(by=["topicId", "similarityScore"], ascending=[True, False])

# Reassign the rank within each topicid group
df['ranking'] = df.groupby('topicId').cumcount() + 1

print(df)

       topicId identifier          docId  ranking  similarityScore systemName
0          401         Q0     FBIS4-9582        1        13.814581    plt8ah2
1          401         Q0    FT932-10861        2        13.342205    plt8ah2
2          401         Q0    FBIS4-18182        3        12.389992    plt8ah2
3          401         Q0     FBIS3-9104        4        12.071641    plt8ah2
4          401         Q0    FT941-15167        5        11.571873    plt8ah2
...        ...        ...            ...      ...              ...        ...
49993      450         Q0    FBIS4-13170      996        10.328714    plt8ah2
49994      450         Q0  LA082190-0080      997        10.327399    plt8ah2
49995      450         Q0  LA123189-0117      998        10.324968    plt8ah2
49996      450         Q0  LA102490-0024      999        10.324179    plt8ah2
49997      450         Q0    FBIS4-11642     1000        10.323837    plt8ah2

[49998 rows x 6 columns]


In [15]:
# Save the cleaned DataFrame back to a file
save_path = "D:\VSCODE PROJECT\IR\cleaned_dataset\cleaned_input.plt8ah2.txt"

df.to_csv(save_path, sep="\t", header=False, index=False)